# DataSens E1_v1 — 04_crud_tests

- Objectifs: démontrer CRUD sur `document` + QA simples
- Prérequis: 03_ingest_sources
- Vérifs: counts, update, delete, doublons
- Guide: docs/GUIDE_TECHNIQUE_E1.md



> Notes:
> - Ouverture de la base SQLite et définition de 4 helpers: CREATE/READ/UPDATE/DELETE.
> - `list_documents()` s’appuie sur pandas pour un affichage rapide.
> - Ordre de test: lister → insérer → relister → mettre à jour → relister → supprimer.
> - But: valider le cycle CRUD avant les analytics.


In [ ]:
# DataSens E1_v1 - 04_crud_tests
# 🛠️ Utilitaires CRUD pour `document` + test
import sqlite3
from pathlib import Path

import pandas as pd

# 1) Ouvrir la base SQLite
a = Path.cwd()
PROJECT_ROOT = a.parents[2] if (a.name.startswith('datasens_')) else a
DB_PATH = PROJECT_ROOT / "datasens" / "datasens.db"
conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

# 2) Petit helper pour lire des SELECT dans un DataFrame (affichage pratique)
def sql_df(query, params=None):
    return pd.read_sql_query(query, conn, params=params)

# 3) CREATE: insérer un document rattaché à un flux (traçabilité)
def add_document(id_flux, titre, texte, langue, date_publication):
    cur.execute(
        "INSERT INTO document (id_flux, titre, texte, langue, date_publication) VALUES (?, ?, ?, ?, ?)",
        (id_flux, titre, texte, langue, date_publication),
    )
    conn.commit()

# 4) READ: lister et lire un document

def list_documents():
    return sql_df("SELECT id_doc, titre, langue, date_publication FROM document;")

def get_document(id_doc):
    return sql_df("SELECT * FROM document WHERE id_doc = ?", params=[id_doc])

# 5) UPDATE: ne modifie que les champs fournis (mise à jour partielle)

def update_document(id_doc, titre=None, texte=None, langue=None, date_publication=None):
    updates, params = [], []
    if titre is not None:
        updates.append("titre = ?"); params.append(titre)
    if texte is not None:
        updates.append("texte = ?"); params.append(texte)
    if langue is not None:
        updates.append("langue = ?"); params.append(langue)
    if date_publication is not None:
        updates.append("date_publication = ?"); params.append(date_publication)
    if updates:
        params.append(id_doc)
        sql = f"UPDATE document SET {', '.join(updates)} WHERE id_doc = ?"
        cur.execute(sql, params)
        conn.commit()

# 6) DELETE: suppression d’un document
def delete_document(id_doc):
    cur.execute("DELETE FROM document WHERE id_doc = ?", (id_doc,))
    conn.commit()


In [ ]:
# 🧪 Test CRUD sur `document`
print("📄 Documents initiaux :")
try:
    display(list_documents())
except Exception:
    pass

print("➕ Ajout…")
add_document(1, "Inflation 2024", "Hausse des prix ressentie", "fr", "2024-09-15")
try:
    display(list_documents())
except Exception:
    pass

print("✏️ Modification…")
# On suppose que l'ID du dernier document est 2 dans le scénario de démo
update_document(2, titre="Inflation 2024 (mise a jour)")
try:
    display(list_documents())
except Exception:
    pass

print("🗑️ Suppression…")
delete_document(2)
try:
    display(list_documents())
except Exception:
    pass
print("CRUD OK")


Using DB: sqlite:///data/tmp/datasens_e1.db
CRUD smoke OK
